In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_absolute_error

In [52]:
df_aq = pd.read_csv("../input/beijing_1701_1803.csv")
df_meo = pd.read_csv("../input/Beijing_historical_meo_grid.csv")
df_station = pd.read_csv("../input/Beijing_AirQuality_Stations.csv")
target_cols = ['PM2.5', 'PM10', 'O3']

In [48]:
stationId = df_aq.stationId.unique()
utc_time = pd.date_range(start='2018-04-30 00:00:00', end='2018-05-01 23:00:00', freq="1H")
df_sample = pd.DataFrame([[i, j] for i in utc_time for j in stationId], columns=['utc_time', 'stationId'])
df_sample

,utc_time,stationId
0,2018-04-30 00:00:00,aotizhongxin_aq
1,2018-04-30 00:00:00,badaling_aq
2,2018-04-30 00:00:00,beibuxinqu_aq
3,2018-04-30 00:00:00,daxing_aq
4,2018-04-30 00:00:00,dingling_aq
5,2018-04-30 00:00:00,donggaocun_aq
6,2018-04-30 00:00:00,dongsi_aq
7,2018-04-30 00:00:00,dongsihuan_aq
8,2018-04-30 00:00:00,fangshan_aq
9,2018-04-30 00:00:00,fengtaihuayuan_aq


In [1]:
%%time

from sklearn.model_selection import KFold, train_test_split, GridSearchCV
import xgboost as xgb
from lightgbm.sklearn import LGBMRegressor

def SMAPE(y_pred, dtrain): 
    y_true = dtrain.get_label()
    return 'SMAPE', np.mean(np.abs((y_pred-y_true) / (y_pred+y_true))) * 2

def MAE(y_pred, dtrain):
    y_true = dtrain.get_label()
    return 'MAE', mean_absolute_error(y_pred, y_true)


dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test, nthread=-1)
param = {
    'max_depth': 3, 
    'learning_rate': 0.1,
    'n_estimators': 200,
    'objective':'gpu:reg:linear',
    'tree_method': 'gpu_hist',
    'eval_metric': 'mae'
}

watchlist = [(dtrain, 'train'), (dtest, 'eval')]
# xgb.train(param, dtrain, num_boost_round=500, evals=watchlist , feval=SMAPE, early_stopping_rounds=10)
gpu_res = {}
xgb.train(param, dtrain, num_boost_round=500, evals=watchlist , early_stopping_rounds=10, evals_result=gpu_res)
# y_pred = xgb_model.predict(X_test)
# print(mean_squared_error(y_test, y_pred))

In [3]:
# xgb.XGBRegressor()

In [ ]:
preds = bst.predict(dtest)